# E-SEIC
Selection of evolutionary instances with constraints for unbalanced datasets

In [1]:
from utils.dataset_utils import get_distribution, k_fold_cross_validation
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection_encapsulation.operator.init_toolbox import init_toolbox_eseic
from instance_selection_encapsulation.operator.metrics import calculate_gmean_mauc, calculate_average_accuracy, calculate_accuracy
from instance_selection_encapsulation.operator.genetic_operator import selTournamentNDCD
from instance_selection_encapsulation.operator.ensemble import vote_result_ensembles, ensemble_individuals
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
import scipy.io as sio  # 从.mat文件中读取数据集
import random
import warnings
import numpy as np
import os
from openpyxl import Workbook

warnings.filterwarnings("ignore")  # 忽略警告
from utils.excel_utils import save_to_excel_2

# 数据的预处理
def data_process(dataset=None, distribution=False):
    datasetname = dataset.DATASETNAME.split('.')[0]
    mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + dataset.DATASETNAME)  # 加载、划分数据集
    x = mat_data['X']
    y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y,
                                                        random_state=RANDOM_SEED)  # 划分数据集
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
    unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
    unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
    weights_train = (1 / counts_train.astype(float)) / np.sum(1 / counts_train.astype(float))  # 计算每个类的权重，用于计算每个类别的权重
    if distribution:
        print(datasetname + f' distribution: {counts_all}')
        print(f'trainset distribution: {counts_train}')
        print(f'testset distribution: {counts_test}')
    model = MLPClassifier(hidden_layer_sizes=(dataset.HIDDEN_SIZE,), max_iter=dataset.MAX_ITER,
                          random_state=RANDOM_SEED, learning_rate_init=dataset.LEARNING_RATE)

    return x_train, x_test, y_train, y_test, weights_train, clone(model)

def main(x_train, y_train, model, balanced_method='random'):
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)  # 个体编码默认全为0
    pop = toolbox.init_population(pop, balanced_method=balanced_method)  # 初始化种群中的个体
    toolbox.evaluate(pop)  # 计算个体的适应度
    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 1):
        offspring = selTournamentNDCD(pop, POPSIZE, tournsize=3)  # 锦标赛选择（1、先根据非支配排序的等级2、再根据拥挤距离）
        offspring = [toolbox.clone(ind) for ind in offspring]
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])  # 单点交叉
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]  # 二进制反转突变
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]  # 二进制反转突变
            del offspring[i].fitness.values, offspring[i + 1].fitness.values
        #############################################################合并、去重#####################################################
        offspring = toolbox.individuals_constraints(offspring)  # 限制每个类至少有一个实例被选择
        pop = pop + offspring  # 种群的合并
        pop, _ = toolbox.remove_duplicates(pop)  # 去重
        while len(pop) < POPSIZE:  # 保证种群大小为POPSIZE
            add_individual = []
            num_add = POPSIZE - len(pop)
            for i in range(0, num_add):
                index = random.randint(0, len(offspring) - 1)  # 在0-len(offspring)范围内随机产生一个索引
                offspring[index] = toolbox.mutate(offspring[index], MR)[0]  # 选择index对应的个体进行突变
                del offspring[index].fitness.values
                add_individual.append(offspring[index])
            add_individual = toolbox.individuals_constraints(add_individual)  # 限制每个类至少有一个实例被选择
            pop = pop + add_individual  # 种群的合并
            pop, _ = toolbox.remove_duplicates(pop)  # 去重
        pop = toolbox.individuals_constraints(pop)  # 限制每个类至少有5个实例被选择
        toolbox.evaluate(pop)  # 计算新种群适应度
        # 非支配排序
        pop, pareto_fronts = toolbox.select(pop, POPSIZE)
    ensemble_classifiers = ensemble_individuals(pop, model, x_train, y_train)
    return ensemble_classifiers

def save_to_excel(data, save_path, filename='avg_results'):
    """
    将列表数据逐行写入Excel文件
    参数:
        data: 二维列表，每个子列表代表一行数据
        filename: 输出的Excel文件名(默认为output.xlsx)
    """
    # 创建一个新的工作簿
    wb = Workbook()
    # 获取活动的工作表
    ws = wb.active
    # 逐行写入数据
    for row in data:
        avg=row[1].tolist()
        avg.insert(0, row[0])
        std=row[2].tolist()     
        std.insert(0, row[0])
        ws.append(avg)
        ws.append(std)
    # 创建 Excel 文件完整路径
    file_path = os.path.join(save_path, filename + ".xlsx")
    # 保存Excel文件
    wb.save(file_path)
    print(f"数据已成功写入到 {file_path}")

## 运行

In [2]:
# DATASETS = [Balance_Scale,Dermatology,Ecoli,Car,Pen_Digits,WallRobot,German,Wine,Nursery,Penbased,USPS,Satellite,Page_Blocks,Shuttle,Contraceptive,Automobile,Ovarian]  # 数据集名称（包含对应的参数配置）
DATASETS = [Penbased,USPS,Satellite,Page_Blocks,Shuttle,Contraceptive,Automobile,Ovarian]  # 数据集名称（包含对应的参数配置）
if __name__ == "__main__":
    save_path = 'C:/Users/Lenovo/Desktop/Non-constraints/'
    columns = ['Gmean', 'MAUC', 'Acc1', 'Acc2', 'Acc3', 'num_ensemble']
    datasets_ensembles_results = [[] for _ in range(len(DATASETS))]
    print("*****************算法开始执行：******************")
    for j, dataset in enumerate(DATASETS):
        x_train, x_test, y_train, y_test, weights_train, model = data_process(dataset=dataset, distribution=False)
        toolbox = init_toolbox_eseic(model, x_train, y_train, weights_train, None, n_splits=N_SPLITS - 2,
                                     random_seed=42)  # 初始化toolbox
        num_run = 40  # 运行次数
        ensembles_results = [[] for _ in range(num_run)]
        for i in range(num_run):
            ensemble_classifiers = main(x_train, y_train, model=model, balanced_method='random')
            vote_pred_prob = vote_result_ensembles(ensemble_classifiers, x_test)  # 默认预测结果是软标签
            vote_pred = np.argmax(vote_pred_prob, axis=1)
            gmean, mauc, recall_per_class = calculate_gmean_mauc(vote_pred_prob, y_test)
            acc1, acc2, acc3 = calculate_accuracy(vote_pred, y_test, weights_train)
            ensembles_results[i] = [gmean, mauc, acc1, acc2, acc3, len(ensemble_classifiers)]
            print(f"第{i + 1}次执行：Gmean：{gmean}，mAUC：{mauc}")
        save_to_excel_2(save_path + dataset.DATASETNAME.split('.')[0] + '/', dataset.DATASETNAME.split('.')[0], columns, ensembles_results)
        ensembles_result_mean = np.mean(ensembles_results, axis=0)
        # 计算ensembles_resultsz中每一列的标准差
        ensembles_result_std = np.std(ensembles_results, axis=0)
        print(f'集成分类结果（平均值）：{ensembles_result_mean}')
        print(f'集成分类结果（标准差）：{ensembles_result_std}')
        datasets_ensembles_results[j] = [dataset.DATASETNAME.split('.')[0], ensembles_result_mean, ensembles_result_std]
    print("*****************算法执行结束！******************")
    # 写入到Excel     
    save_to_excel(datasets_ensembles_results,save_path)         

*****************算法开始执行：******************
第1次执行：Gmean：0.981858，mAUC：0.999026
第2次执行：Gmean：0.962979，mAUC：0.998905
第3次执行：Gmean：0.964934，mAUC：0.998893
第4次执行：Gmean：0.978931，mAUC：0.998088
第5次执行：Gmean：0.978023，mAUC：0.998481
第6次执行：Gmean：0.964119，mAUC：0.99912
第7次执行：Gmean：0.964119，mAUC：0.998936
第8次执行：Gmean：0.954037，mAUC：0.998022
第9次执行：Gmean：0.959813，mAUC：0.998773
第10次执行：Gmean：0.978535，mAUC：0.998841
第11次执行：Gmean：0.972621，mAUC：0.998586
第12次执行：Gmean：0.972621，mAUC：0.999025
第13次执行：Gmean：0.969214，mAUC：0.999403
第14次执行：Gmean：0.975618，mAUC：0.998083
第15次执行：Gmean：0.978644，mAUC：0.998538
第16次执行：Gmean：0.966559，mAUC：0.998506
第17次执行：Gmean：0.964934，mAUC：0.998947
第18次执行：Gmean：0.978841，mAUC：0.9985
第19次执行：Gmean：0.955663，mAUC：0.998557
第20次执行：Gmean：0.975618，mAUC：0.9988
第21次执行：Gmean：0.981858，mAUC：0.998016
第22次执行：Gmean：0.966041，mAUC：0.99858
第23次执行：Gmean：0.981858，mAUC：0.999002
第24次执行：Gmean：0.967908，mAUC：0.99871
第25次执行：Gmean：0.978931，mAUC：0.998236
第26次执行：Gmean：0.965695，mAUC：0.998422
第27次执行：Gmean：0.963589，mAUC：0.998465
第